In [1]:
from IPython.display import display
import mne
from components.dataset import Dataset_Builder
import pickle
import os
import pandas as pd
import numpy as np
dataset_path = "data"
dataset = Dataset_Builder(dataset_path=dataset_path)\
            .with_sampling_rate(125)\
            .with_marker(start_minute=1, stop_minute=3, segment=6)\
            .build()

Found: 55 files
Mean:20.509090909090908, Std:6.149272617066855
Non-stressed:16
Stressed:19
Neutral:20
marker.shape=(20, 3)


In [10]:
def save(data, filename):
    with open(f'data/EEG_numpy_baseline/{filename}.pickle', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
def load(filename):
    with open(f'data/EEG_numpy_baseline/{filename}.pickle', 'rb') as handle:
        data = pickle.load(handle)
    return data

In [4]:
def to_raw(df):
    ch_names = list(df.columns)
    ch_types = ['eeg'] * len(ch_names)
    # https://mne.tools/stable/generated/mne.create_info.html
    info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=125)

    df = df.T  #mne looks at the tranpose() format
    df *= 1e-6  #convert from uVolts to Volts (mne assumes Volts data)

    raw = mne.io.RawArray(df,info=info,verbose='CRITICAL')
    raw.set_montage('standard_1020')
    return raw

In [5]:
def get_markers():
    sampling_rate = 125 #Hz
    # 15/60 = 0.25
    step_minutes = np.arange(0.5, 0.6,0.25)
    print(f"{step_minutes=}")
    step_minutes = np.expand_dims(step_minutes * sampling_rate * 60, axis=1)
    markers = np.concatenate( [step_minutes, np.zeros( step_minutes.shape ), np.ones( step_minutes.shape ) ], axis=1  ).astype(np.int64)
    return markers
markers = get_markers()
markers.shape
markers

step_minutes=array([0.5])


array([[3750,    0,    1]])

In [19]:
for f in dataset.files:
    _,name = os.path.split(f)
    name,ext = os.path.splitext(name)
    df = pd.read_csv(f, index_col='Index')
    # print(df.shape)
    raw = to_raw(df)
    epochs = mne.Epochs(raw, markers, tmin=-1, tmax=5*60, baseline=(None,0), preload=True)
    data = epochs.get_data()[0]
    print(f, data[:, 126:].shape)
    assert data[:, 126:].shape == (16,37500)
    save(data[:, 126:], name)
    # break
    
    # break
    # df = df.loc[0.5*125*60: (5.5*125*60)-1]
    # df = df.T
    # df = df * 1e-6
    # data = df.to_numpy()
    # print(name, data.shape)
    # save(data, name)
    # print(f, df.shape, df.shape[1]/125)
    # break

Not setting metadata
1 matching events found
Setting baseline interval to [-1.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 37626 original time points ...
0 bad epochs dropped
data/EEG/001.csv (16, 37500)
Not setting metadata
1 matching events found
Setting baseline interval to [-1.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 37626 original time points ...
0 bad epochs dropped
data/EEG/002.csv (16, 37500)
Not setting metadata
1 matching events found
Setting baseline interval to [-1.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 37626 original time points ...
0 bad epochs dropped
data/EEG/003.csv (16, 37500)
Not setting metadata
1 matching events found
Setting baseline interval to [-1.0, 0.0] sec
Applying baseline correction (mode: mean)
0